In [2]:
import pandas as pd
import numpy as np
import nltk
import regex
import collections, re
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
from sklearn.cluster import KMeans
from tqdm import tqdm
from gensim.models import Word2Vec 
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.cluster import AffinityPropagation
from sklearn.manifold import TSNE
#import spacy
from sklearn.preprocessing import normalize
import sys

In [3]:
reddit = pd.read_csv('nationalservicesg_combineddata.csv')
reddit = reddit['content']
google = pd.read_csv('all-camps-google-reviews.csv')
google = google['Content']
combined = [reddit, google]
reddit_google = pd.concat(combined)
reddit_google = reddit_google.dropna()
reddit_google = reddit_google.to_frame( name = 'content')
reddit_google.to_csv('reddit-google.csv')

In [4]:
def pairwise(iterable):
    """s -> (s0,s1), (s1,s2), (s2, s3), ..."""
    a, b = itertools.tee(iterable)
    next(b, None)
    return ((a, b) if a < b else (b, a) for a, b in zip(a, b))

In [5]:
import itertools
from collections import Counter
counter = Counter()
test = 'Unlike what the other reviews may suggest, people here were adequately friendly(like anywhere else in sg). The place is really clean and efficient'.split()
#word_tokenize(test)
counter = collections.Counter(pairwise(test))
#test.split()

In [6]:
text = []
for index, review in enumerate(reddit_google['content']):
    #print(index)
    r = review.split()
    counter.update(pairwise(r))

In [7]:
df = pd.DataFrame.from_dict(counter, orient='index').reset_index()
for pairs in df['index']:
    pairs = ' '.join(pairs)
df

,index,0
0,"(Unlike, what)",2
1,"(the, what)",10
2,"(other, the)",13
3,"(other, reviews)",2
4,"(may, reviews)",2
...,...,...
16701,"(get, rewarded)",1
16702,"(for, rewarded)",1
16703,"(exercising, for)",1
16704,"(Life, of)",1


In [8]:
df = df.rename(columns={'index':'pair', 0:'freq'})
df

,pair,freq
0,"(Unlike, what)",2
1,"(the, what)",10
2,"(other, the)",13
3,"(other, reviews)",2
4,"(may, reviews)",2
...,...,...
16701,"(get, rewarded)",1
16702,"(for, rewarded)",1
16703,"(exercising, for)",1
16704,"(Life, of)",1


In [9]:
type(df.iloc[0]['pair'])

tuple

In [10]:
tokenizer = dict()
reddit_google['content'] = reddit_google['content'].apply(
    lambda content: [tokenizer.setdefault(named_entitie, len(tokenizer))
                            for named_entitie in content.split(' ')])
reddit_google.head()


,content
0,"[0, 1, 2, 3, 4, 5]"
1,"[6, 7, 5]"
2,"[8, 9, 10, 11, 12, 13, 14, 15, 8, 16, 17, 18, ..."
3,"[32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 5]"
4,"[43, 44, 45, 46, 47, 5]"


In [11]:
pairs_df = reddit_google['content'].apply(lambda named_entities: list(itertools.combinations(named_entities, 2)))
pairs_df = pairs_df[pairs_df.apply(len) > 0]
pairs_df = pd.DataFrame(np.concatenate(pairs_df.values), columns=['named_entity_1', 'named_entity_2'])
pairs_df.head()

,named_entity_1,named_entity_2
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5


In [15]:
NAMED_ENTITIES_CO_OCCURENCE_THRESHOLD = 50

edges_df = pairs_df.groupby(['named_entity_1', 'named_entity_2']).size().reset_index(name='weight')
edges_df = edges_df[edges_df['weight'] > NAMED_ENTITIES_CO_OCCURENCE_THRESHOLD]
edges_df[['named_entity_1', 'named_entity_2', 'weight']].to_csv('edges.csv', header=False, index=False, sep=' ')
edges_df.head()

,named_entity_1,named_entity_2,weight
20,3,19,51
29,3,34,134
33,3,44,83
48,3,73,72
886,5,5,53


In [16]:
edges_df.sort_values(by = 'weight', ascending = False)

,named_entity_1,named_entity_2,weight
22588,34,34,1997
31263,44,34,1638
22597,34,44,1579
31269,44,44,1498
31294,44,73,1247
...,...,...,...
154473,383,36,51
31739,44,610,51
149587,360,36,51
31838,44,732,51


In [17]:
nodes = pd.DataFrame(list(tokenizer.items()), columns = ['word','number'])
nodes

,word,number
0,Thigh,0
1,muscle,1
2,pains,2
3,during,3
4,NS.,4
...,...,...
5450,year~,5450
5451,whete,5451
5452,rewarded,5452
5453,exercising,5453


In [18]:
edges_df.to_csv('edges.csv')
nodes.to_csv('nodes.csv')